# redMaPPer sdssWISEphot

The goal of this notebook is to document how we gather SDSS/DR14 *ugriz* and unWISE *W1-W4* (forced) photometry for the parent sample of redMaPPer central galaxies.

The input catalog is
* $REDMAPPER_DIR/v6.3.1/dr8_run_redmapper_v6.3.1_lgt5_catalog.fit

and the output (row-matched) catalog containing *just* the SDSS/unWISE photometry is
* $REDMAPPER_DIR/v6.3.1/redmapper-v6.3.1-lgt5-sdssWISEphot-dr14.fits

In [1]:
import os
import numpy as np

from astropy.table import Table, Column
import fitsio

### Read the data and transform the PhotoIDs in the redMaPPer catalog to ObjIDs.

In [2]:
def read_redmapper(rmversion='v6.3.1', index=None):
    """Read the parent redMaPPer catalog.
    
    """
    rmfile = os.path.join( os.getenv('REDMAPPER_DIR'), rmversion, 
                          'dr8_run_redmapper_{}_lgt5_catalog.fit'.format(rmversion) )
    if index is not None:
        rm = Table(fitsio.read(rmfile, ext=1, upper=True, rows=index))
    else:
        rm = Table(fitsio.read(rmfile, ext=1, upper=True))
    print('Read {} objects from {}'.format(len(rm), rmfile))

    return rm

In [3]:
rmversion = 'v6.3.1'

In [4]:
rm = read_redmapper(rmversion=rmversion)
nobj = len(rm)
rm

Read 396047 objects from /Users/ioannis/research/data/redmapper/v6.3.1/dr8_run_redmapper_v6.3.1_lgt5_catalog.fit


MEM_MATCH_ID,RA,DEC,MODEL_MAG [5],MODEL_MAGERR [5],IMAG,IMAG_ERR,ZRED,ZRED_E,ZRED_CHISQ,BCG_SPEC_Z,Z_SPEC_INIT,Z_INIT,Z,LAMBDA_CHISQ,LAMBDA_CHISQ_E,LAMBDA_ZRED,LAMBDA_ZRED_E,R_LAMBDA,SCALEVAL,MASKFRAC,C_LAMBDA [4],C_LAMBDA_ERR [4],MAG_LAMBDA_ERR [5],CHISQ,Z_LAMBDA,Z_LAMBDA_E,Z_LAMBDA_NITER,EBV_MEAN,PHOTOID,LNLAMLIKE,LNBCGLIKE,LNLIKE,PZBINS [21],PZ [21],NCROSS,RMASK,RA_ORIG,DEC_ORIG,W,DLAMBDA_DZ,DLAMBDA_DZ2,DLAMBDAVAR_DZ,DLAMBDAVAR_DZ2,LAMBDA_CHISQ_C,LAMBDA_CHISQ_CE,NCENT,NCENT_GOOD,RA_CENT [5],DEC_CENT [5],ID_CENT [5],LAMBDA_CHISQ_CENT [5],ZLAMBDA_CENT [5],P_BCG [5],P_CEN [5],Q_CEN [5],P_FG [5],Q_MISS,P_SAT [5],P_C [5],BCG_ILUM,ILUM,Z_LAMBDA_RAW,Z_LAMBDA_E_RAW,LIM_EXPTIME,LIM_LIMMAG,LIM_LIMMAG_HARD
int32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,int64,float32,float32,float32,float32,float32,int16,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,int16,int16,float64,float64,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
1,239.583329069,27.2334129178,17.7673 .. 13.6197,0.0495718 .. 0.00515177,13.979,0.00269815,0.0976015,0.00598646,17.5295,0.0908067,0.0908067,0.0976015,0.0995999,169.838,4.18703,167.9,3.82714,1.11175,1.01137,0.012301,1.91101 .. 0.340558,0.00478467 .. 0.00015905,0.0047196 .. 0.0,17.1603,0.0948307,0.00458178,1,0.0156272,39270301200850016,417.533,3.98176,421.515,0.0689421 .. 0.120719,6.66542e-06 .. 8.29784e-06,1,1.66762,239.583329069,27.2334129178,1.09264,22.1429,-1019.21,431.903,85493.5,169.368,1.24138,5,2,239.583329069 .. 239.559634173,27.2334129178 .. 27.2721026726,25404293 .. 25404056,169.838 .. 165.795,0.0964975 .. 0.0,1.0 .. 1.0,0.874838 .. 5.36843e-07,0.752766 .. 4.61934e-07,0.000188751 .. 0.00250284,-4.7971,0.124973 .. 0.997497,0.843622 .. 3.31047e-06,7.45976,115.15,0.0964975,0.0015025,122.966,21.1772,21.2911
2,250.082548387,46.7115313536,19.7931 .. 15.8147,0.144956 .. 0.0193181,16.2903,0.0075439,0.263556,0.00704976,16.1534,-1.0,-1.0,0.233135,0.233327,199.539,5.29983,173.621,5.04967,1.14817,0.995171,0.0,1.80072 .. 0.32185,0.116592 .. 0.00175495,0.115213 .. 0.0,22.2347,0.232764,0.00615558,1,0.0268378,14530301500570247,382.432,1.15807,383.59,0.196232 .. 0.269297,3.38395e-07 .. 5.93477e-06,1,1.72225,250.086476355,46.7122681373,1.25965,7.30459,-56.4033,228.875,22720.9,199.594,0.463512,5,2,250.082548387 .. 250.107576522,46.7115313536 .. 46.7094645939,3352952 .. 3352965,199.539 .. 210.08,0.231425 .. 0.0,1.0 .. 1.0,0.985673 .. 3.25354e-06,0.633526 .. 2.09117e-06,0.0115227 .. 0.129715,-2.05149,0.00280448 .. 0.870282,0.639414 .. 5.85322e-06,5.72643,132.716,0.231425,0.00207641,94.0304,21.0016,21.2911
3,197.872957171,-1.34111627953,20.4303 .. 15.0278,0.495924 .. 0.0141083,15.3854,0.00549334,0.193564,0.00764036,32.7315,-1.0,-1.0,0.165282,0.185414,164.712,4.24162,157.558,3.99728,1.10496,1.00655,0.00508906,1.83547 .. 0.321014,0.0797838 .. 0.00102912,0.0790554 .. 0.0,34.0984,0.182398,0.00393653,1,0.0221436,23340301600130074,372.586,2.67244,375.259,0.159875 .. 0.204921,8.13114e-06 .. 1.31497e-05,1,1.65743,197.876240818,-1.34142305978,1.46013,2.03778,-741.054,128.44,24408.3,164.286,0.620295,5,3,197.872957171 .. 197.886363462,-1.34111627953 .. -1.32545764765,9222531 .. 9222794,164.712 .. 160.685,0.184493 .. 0.0,1.0 .. 1.0,0.660098 .. 0.000373478,0.653537 .. 0.000369766,0.000265913 .. 0.0163708,-18.7437,0.339636 .. 0.983256,0.98502 .. 0.0358702,8.11466,118.677,0.184493,0.00164668,85.9083,20.9194,21.2911
4,230.600091876,27.7143690526,17.7457 .. 14.1279,0.0249274 .. 0.00438643,14.449,0.00252544,0.0758894,0.00859367,3.21969,0.069041,-1.0,0.0535868,0.0786437,133.256,4.30238,133.333,3.49261,1.0591,1.0523,0.0482489,1.87192 .. 0.316875,0.00237442 .. 0.00010312,0.00233411 .. 0.0,2.20879,0.0774831,0.00579991,2,0.0156272,45120301202090548,370.72,

#### Convert the PhotoID in the redMaPPer catalog to ObjID using utilities written by Erin Sheldon.

In [5]:
def unpack_photoid(photoid):
    """Unpack the SDSS photoID into run, rerun, camcol, field, and id.
    
    The heart of this code is adopted from Erin Sheldon's 
    sdsspy.util.photoid_extract.

    """    
    import numpy as np
    
    pvals = [13,9,8,4,0]

    ten = np.array(10, dtype='i8')

    run = photoid // ten**pvals[0]
    rerun = photoid // ten**pvals[1] - run*ten**(pvals[0]-pvals[1])

    camcol = \
        photoid // ten**pvals[2] - \
        run * ten**(pvals[0]-pvals[2]) - \
        rerun * ten**(pvals[1]-pvals[2])

    field = \
        photoid // ten**pvals[3] - \
        run * ten**(pvals[0]-pvals[3]) - \
        rerun * ten**(pvals[1]-pvals[3]) - \
        camcol * ten**(pvals[2]-pvals[3])

    id = \
        photoid // ten**pvals[4] - \
        run * ten**(pvals[0]-pvals[4]) - \
        rerun * ten**(pvals[1]-pvals[4]) - \
        camcol * ten**(pvals[2]-pvals[4]) - \
        field * ten**(pvals[3]-pvals[4])

    return run, rerun, camcol, field, id

In [6]:
def unpack_objid(objid, everything=False):
    """Unpack the SDSS ObjID into run, rerun, camcol, field, and id.
    
    The heart of this code is adopted from Erin Sheldon's 
    sdsspy.util.objid_extract.

    """    
    masks = {
        'sky_version': 0x7800000000000000,
        'rerun': 0x07FF000000000000,
        'run': 0x0000FFFF00000000,
        'camcol': 0x00000000E0000000,
        'first_field': 0x0000000010000000,
        'field': 0x000000000FFF0000,
        'id': 0x000000000000FFFF
    }

    run = (objid & masks['run']) >> 32
    rerun = (objid & masks['rerun']) >> 48
    camcol = (objid & masks['camcol']) >> 29
    field = (objid & masks['field']) >> 16
    id = (objid & masks['id']) >> 0
    sky_version = (objid & masks['sky_version']) >> 59
    first_field = (objid & masks['first_field']) >> 28
    
    if everything:
        return run, rerun, camcol, field, id, first_field, sky_version
    else:
        return run, rerun, camcol, field, id

In [7]:
def get_objid(run, rerun, camcol, field, id, skyversion=2):
    """Convert run, rerun, camcol, field, and id to an ObjID used 
    by the SDSS CAS.
    
    The heart of this code is adopted from Erin Sheldon's 
    sdsspy.util.get_odjid.

    """    
    import numpy as np
    
    dt = 'i8'
    sky    = np.array(skyversion, ndmin=1, dtype=dt, copy=False)
    run    = np.array(run, ndmin=1, dtype=dt, copy=False)
    rerun  = np.array(rerun, ndmin=1, dtype=dt, copy=False)
    camcol = np.array(camcol, ndmin=1, dtype=dt, copy=False)
    field  = np.array(field, ndmin=1, dtype=dt, copy=False)
    id     = np.array(id, ndmin=1, dtype=dt, copy=False)
    
    objid = np.zeros(run.size, dtype=dt)

    first_field = np.zeros(run.size, dtype=dt)
    objid = (objid
             | (sky << 59)
             | (rerun << 48)
             | (run << 32)
             | (camcol << 29)
             | (first_field << 28)
             | (field << 16)
             | (id << 0) )

    return objid    

In [8]:
photoid = rm['PHOTOID'].data
run, rerun, camcol, field, id = unpack_photoid(photoid)
objid = get_objid(run, rerun, camcol, field, id)

#### [1] Match using the original ObjID.

Write out an ASCII file with the original ObjID, upload it to CasJobs (naming the table "redmapper_originalobjid_in", execute the following SQL query (using the DR14 Context), and download the resulting table, "redmapper_originalobjid_out".

```SQL
SELECT
  m.redmapper_id,
  m.redmapper_objid,
  p.objid,
  p.ra,
  p.dec,
  p.run,
  p.rerun,
  p.camcol
INTO mydb.redmapper_originalobjid_out FROM mydb.redmapper_originalobjid_in AS m
  JOIN dr14.PhotoObj AS p ON m.redmapper_objid=p.objid
ORDER BY m.redmapper_id
```

In [9]:
casfile = os.path.join( os.getenv('REDMAPPER_DIR'), rmversion, 'casjobs', 
                       'redmapper_originalobjid_in.txt'.format(rmversion) )
if os.path.isfile(casfile):
    print('CasJobs file {} exists.'.format(casfile))
else:
    print('Writing {}'.format(casfile))
    np.savetxt( casfile, np.vstack( [ [rm['MEM_MATCH_ID']], [objid] ] ).T, 
               header='redmapper_id redmapper_objid', fmt=('%i', '%i') )

CasJobs file /Users/ioannis/research/data/redmapper/v6.3.1/casjobs/redmapper_originalobjid_in.txt exists.


#### [2] Gather photometry of missing objects.

Read the output of the previous step and determine which objects are missing.  (In our case ObjIDs didn't match for just 43/396047, or 0.01% objects!)  

Then write out and upload a new CasJobs file with the RA,Dec coordinates of the missing objects (calling the table "redmapper_missing_in"), execute the following SQL query (again using the DR14 Context), and download the resulting table, "redmapper_missing_out".

Note that we write out an XML table because CasJobs does not maintain the double precision of the RA,Dec coordinates in the ASCII file.

Also note that we use a generous 5 arcsec search radius.

```SQL
SELECT
  m.redmapper_id,
  m.redmapper_objid,
  m.redmapper_ra,
  m.redmapper_dec,
  p.objid,
  p.ra,
  p.dec,
  p.run,
  p.rerun,
  p.camcol
INTO mydb.redmapper_missing_out FROM mydb.redmapper_missing_in AS m
  OUTER APPLY dbo.fGetNearestObjEq( m.redmapper_ra, m.redmapper_dec, 0.01666) AS n 
  LEFT JOIN dr14.PhotoObj AS p ON n.objid=p.objid
ORDER BY m.redmapper_id
```

In [10]:
originalfile = os.path.join( os.getenv('REDMAPPER_DIR'), rmversion, 'casjobs',
                            'redmapper_originalobjid_out.fits' )
oo = Table(fitsio.read(originalfile, ext=1, upper=True))
print('Read {} objects from {}'.format(len(oo), originalfile))

Read 396004 objects from /Users/ioannis/research/data/redmapper/v6.3.1/casjobs/redmapper_originalobjid_out.fits


In [11]:
notmissing = np.where(np.in1d(rm['MEM_MATCH_ID'], oo['REDMAPPER_ID']))[0]
missing = np.where(~np.in1d(rm['MEM_MATCH_ID'], oo['REDMAPPER_ID']))[0]
print('The following {} objects did not match on their original ObjID:'.format(len(missing)))
rm[missing]

The following 43 objects did not match on their original ObjID:


MEM_MATCH_ID,RA,DEC,MODEL_MAG [5],MODEL_MAGERR [5],IMAG,IMAG_ERR,ZRED,ZRED_E,ZRED_CHISQ,BCG_SPEC_Z,Z_SPEC_INIT,Z_INIT,Z,LAMBDA_CHISQ,LAMBDA_CHISQ_E,LAMBDA_ZRED,LAMBDA_ZRED_E,R_LAMBDA,SCALEVAL,MASKFRAC,C_LAMBDA [4],C_LAMBDA_ERR [4],MAG_LAMBDA_ERR [5],CHISQ,Z_LAMBDA,Z_LAMBDA_E,Z_LAMBDA_NITER,EBV_MEAN,PHOTOID,LNLAMLIKE,LNBCGLIKE,LNLIKE,PZBINS [21],PZ [21],NCROSS,RMASK,RA_ORIG,DEC_ORIG,W,DLAMBDA_DZ,DLAMBDA_DZ2,DLAMBDAVAR_DZ,DLAMBDAVAR_DZ2,LAMBDA_CHISQ_C,LAMBDA_CHISQ_CE,NCENT,NCENT_GOOD,RA_CENT [5],DEC_CENT [5],ID_CENT [5],LAMBDA_CHISQ_CENT [5],ZLAMBDA_CENT [5],P_BCG [5],P_CEN [5],Q_CEN [5],P_FG [5],Q_MISS,P_SAT [5],P_C [5],BCG_ILUM,ILUM,Z_LAMBDA_RAW,Z_LAMBDA_E_RAW,LIM_EXPTIME,LIM_LIMMAG,LIM_LIMMAG_HARD
int32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,int64,float32,float32,float32,float32,float32,int16,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,int16,int16,float64,float64,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
22471,155.76300038,57.5615379001,22.7153 .. 17.4953,0.880832 .. 0.0297911,17.8287,0.0128143,0.317341,0.0148829,1.73192,0.334343,0.334343,0.317341,0.340901,22.2765,2.50593,13.4133,1.9087,0.740575,0.99349,0.0,1.5387 .. 0.359179,0.568845 .. 0.00645501,0.54381 .. 0.0,1.31372,0.349267,0.0179885,1,0.137114,22430301602360137,22.8763,-0.565203,22.3111,0.255227 .. 0.443307,3.93495e-07 .. 2.60973e-06,1,1.11086,155.76300038,57.5615379001,1.43552,8.17327,-258.758,53.0877,-1491.2,22.2765,0.0,5,1,155.76300038 .. 155.746162377,57.5615379001 .. 57.589205812,7638018 .. 7638050,22.2765 .. 0.0,0.334937 .. 0.0,1.0 .. 1.0,0.998829 .. 1.11465e-06,0.791207 .. 8.82951e-07,8.18993e-05 .. 0.487252,-2.90574,0.00108945 .. 0.512747,0.791942 .. 4.24769e-06,3.30616,14.6403,0.334937,0.012719,109.83,21.1386,21.2911
30043,224.46574546,56.9384951721,23.0463 .. 18.2324,0.552041 .. 0.0325983,18.4954,0.0148845,0.385174,0.0133786,1.61488,-1.0,-1.0,0.385174,0.357582,26.5784,3.0714,13.7795,2.26598,0.767195,1.00871,0.0130764,1.23034 .. 0.380735,0.24375 .. 0.0063605,0.227909 .. 0.0,2.95253,0.364688,0.0166829,1,0.00991823,61220301500120272,19.3478,0.656011,20.0038,0.282366 .. 0.44701,4.21125e-05 .. 2.415e-06,1,1.15079,224.46574546,56.9384951721,1.07564,10.7551,-1062.49,39.5108,-7499.6,26.4808,0.417877,5,3,224.46574546 .. 224.497742729,56.9384951721 .. 56.9477772031,46017769 .. 46017813,26.5784 .. 21.651,0.351733 .. 0.0,1.0 .. 1.0,0.910682 .. 0.00399157,0.078888 .. 0.00034577,0.0349133 .. 0.298359,0.0610055,0.0544051 .. 0.69765,0.0795031 .. 0.00037842,2.01451,17.6802,0.351733,0.0106678,117.391,21.2009,21.2911
54556,131.248874873,57.0979488789,21.6514 .. 18.1365,0.423332 .. 0.0593842,18.5615,0.0242982,0.469142,0.0118804,7.31877,0.463189,0.463189,0.469142,0.442732,32.8329,7.27363,26.0139,4.38038,0.800317,1.91532,0.0748722,2.57605 .. 0.37638,1.20851 .. 0.0329465,1.0851 .. 0.0,9.62118,0.451084,0.0161992,2,0.137114,42080301101820335,13.3664,1.84391,15.2104,0.351004 .. 0.551164,3.6402e-06 .. 2.85553e-06,2,1.20048,131.248874873,57.0979488789,1.32246,4.51707,-187.842,671.52,-47052.4,32.8329,0.0,5,1,131.248874873 .. 131.262179008,57.0979488789 .. 57.0860657396,27659406 .. 27659226,32.8329 .. 0.0,0.437941 .. 0.0,1.0 .. 1.0,0.995089 .. 0.00019121,0.716645 .. 0.000137706,0.00179335 .. 0.236449,-1.84476,0.00311772 .. 0.76336,0.719189 .. 0.000491884,3.2059,16.2029,0.437941,0.0129837,76.6538,20.6913,21.2911
63828,229.866575853,11.6052448738,25.1007 .. 18.0913,2.10731 .. 0.0474661,18.5117,0.0217141,0.413849,0.0107661,3.3345,0.435419,-1.0,0.456392,0.441699,16.8152,3.06125,6.28926,1.77933,0.700068,1.11486,0.0,1.47034 .. 0.394161,0.5954 .. 0.0163549,0.543426 .. 0.0,5.36831,0.394102,0.0211677,7,0.0248643,39960301501700546,12.0433,-3.62422,8.41905,0.296741 .. 0.491462,0.0149743 .. 2.31

In [12]:
print('For the record, the missing (original) ObjIDs are:')
print(objid[missing])

For the record, the missing (original) ObjIDs are:
[1237655109445091465 1237671769071681808 1237663546367934799
 1237662637981565474 1237678919139590184 1237657858218524958
 1237654381441122664 1237659149922140393 1237658312402404334
 1237661386541498820 1237679454922866959 1237659133280059619
 1237655369292448339 1237651714797535660 1237678860072453257
 1237661150302765337 1237663787424940393 1237651537650254379
 1237662198821815431 1237658609294246073 1237654398081237216
 1237671766925377740 1237667536402907288 1237667916489490609
 1237651495224410309 1237667141793153337 1237651191363404083
 1237657770172416324 1237651271366607031 1237653613183631782
 1237668365315540259 1237663547439448844 1237651271887618529
 1237651250411930107 1237661435381023131 1237648705122206096
 1237657628994044375 1237651539778011687 1237648720677109765
 1237651273497706834 1237680530276876968 1237661417669919465
 1237665128546042063]


In [13]:
xml = True
if xml:
    suffix = 'xml'
else:
    suffix = 'txt'

In [14]:
missingfile = os.path.join( os.getenv('REDMAPPER_DIR'), rmversion, 'casjobs',
                           'redmapper_missing_in.{}'.format(suffix) )
if os.path.isfile(missingfile):
    print('CasJobs file {} exists.'.format(missingfile))
else:
    print('Writing {}'.format(missingfile))
    if xml:
        from astropy.io import votable
        out = Table()
        out.add_column(Column(data=rm['MEM_MATCH_ID'][missing], name='redmapper_id'))
        out.add_column(Column(data=objid[missing], name='redmapper_objid'))
        out.add_column(Column(data=rm['RA'][missing], name='redmapper_ra'))
        out.add_column(Column(data=rm['DEC'][missing], name='redmapper_dec'))
        out_votable = votable.from_table(out)
        votable.writeto(out_votable, missingfile)
    else:
        np.savetxt( missingfile, np.vstack( [ [rm['MEM_MATCH_ID'][missing]], [objid[missing]], 
                                             [rm['RA'][missing]], [rm['DEC'][missing]] ] ).T, 
                   header='redmapper_id redmapper_objid redmapper_ra redmapper_dec', 
                   fmt=('%i', '%i', '%16.10f', '%16.10f') )

CasJobs file /Users/ioannis/research/data/redmapper/v6.3.1/casjobs/redmapper_missing_in.xml exists.


#### [3] Attempt to do a by-hand search for the last few missing objects.

After completing the previous step, we find that we are still missing photometry for 8/43 objects, so we use [Navigate](http://skyserver.sdss.org/dr14/en/tools/chart/navi.aspx) to try to find them.  

Although we find that we can recover ObjIDs for all of them, 4 have a *"WARNING: This object's photometry may be unreliable. See the photometric flags below."*, and so these objects will not be matched when we execute our final SQL query in step [4].

At the end of this step we gather all the updated ObjIDs.

In [15]:
missingoutfile = os.path.join( os.getenv('REDMAPPER_DIR'), rmversion, 'casjobs', 
                              'redmapper_missing_out.fits' )
mm = Table(fitsio.read(missingoutfile, ext=1, upper=True))
print('Read {} objects from {}'.format(len(mm), missingoutfile))

Read 43 objects from /Users/ioannis/research/data/redmapper/v6.3.1/casjobs/redmapper_missing_out.fits


In [16]:
stillnotmissing = np.where(mm['OBJID'] > 0)[0]
stillmissing = np.where(mm['OBJID'] < 0)[0]
print('The following {} objects are still missing ObjIDs:'.format(len(stillmissing)))
mm[stillmissing]

The following 8 objects are still missing ObjIDs:


REDMAPPER_ID,REDMAPPER_OBJID,REDMAPPER_RA,REDMAPPER_DEC,OBJID,RA,DEC,RUN,RERUN,CAMCOL
int32,int64,float64,float64,int64,float64,float64,int16,int16,uint8
85666,1237678919139590184,20.2430974665,11.6518175043,-9223372036854775808,nan,nan,-32768,-32768,128
145282,1237659149922140393,221.771688092,54.2201103797,-9223372036854775808,nan,nan,-32768,-32768,128
163434,1237658312402404334,176.835810591,60.0896418644,-9223372036854775808,nan,nan,-32768,-32768,128
202893,1237651537650254379,236.566132652,54.9171156958,-9223372036854775808,nan,nan,-32768,-32768,128
365649,1237651271366607031,201.20658089,65.5829127834,-9223372036854775808,nan,nan,-32768,-32768,128
398209,1237651271887618529,130.075945026,52.9345376097,-9223372036854775808,nan,nan,-32768,-32768,128
425203,1237657628994044375,191.338957026,51.911202363,-9223372036854775808,nan,nan,-32768,-32768,128
469002,1237648720677109765,141.661187499,-0.515480572129,-9223372036854775808,nan,nan,-32768,-32768,128


The original ("old") and updated ("new") ObjIDs were found by doing a nearest search query using Navigate.  The objects flagged "BAD" have a "this object's photometry may be unreliable" warning.

In [17]:
oldandnew = [ 
    (1237678919139590184, 1237678858480255092, 'OK'),
    (1237659149922140393, 1237659149922140394, 'BAD'),
    (1237658312402404334, 1237658312402404326, 'OK'),
    (1237651537650254379, 1237651537650254378, 'OK'),
    (1237651271366607031, 1237651271366607032, 'BAD'),
    (1237651271887618529, 1237651700840923446, 'BAD'),
    (1237657628994044375, 1237657628994044078, 'OK'),
    (1237648720677109765, 1237648720677044442, 'BAD')
]             

In [18]:
newobjid = np.zeros_like(objid)
newobjid[notmissing] = objid[notmissing]
newobjid[missing[stillnotmissing]] = mm['OBJID'][stillnotmissing].data
for indx, oandn in zip( missing[stillmissing], oldandnew ):
    newobjid[indx] = oandn[1]
assert(np.sum(newobjid == 0) == 0)

#### [4] Gather the final SDSS + unWISE photometry.

Finally, we construct one more CasJobs input file with the updated ObjIDs and request the full set of SDSS (*ugriz*) and unWISE (*W1W2*) forced photometry by executing the following query.  We call the input table "redmapper_newobjid" and the output (FITS) table "redmapper_sdssWisephot.fits".

```SQL
SELECT
  m.redmapper_id,
  m.redmapper_objid,
  p.objid,
  p.ra,
  p.dec,
  p.run,
  p.rerun,
  p.camcol,
  p.field,
  p.type,
  p.petroflux_u,
  p.petroflux_g,
  p.petroflux_r,
  p.petroflux_i,
  p.petroflux_z,
  p.petrofluxivar_u,
  p.petrofluxivar_g,
  p.petrofluxivar_r,
  p.petrofluxivar_i,
  p.petrofluxivar_z,
  p.modelflux_u,
  p.modelflux_g,
  p.modelflux_r,
  p.modelflux_i,
  p.modelflux_z,
  p.modelfluxivar_u,
  p.modelfluxivar_g,
  p.modelfluxivar_r,
  p.modelfluxivar_i,
  p.modelfluxivar_z,
  p.cmodelflux_u,
  p.cmodelflux_g,
  p.cmodelflux_r,
  p.cmodelflux_i,
  p.cmodelflux_z,
  p.cmodelfluxivar_u,
  p.cmodelfluxivar_g,
  p.cmodelfluxivar_r,
  p.cmodelfluxivar_i,
  p.cmodelfluxivar_z,
  p.extinction_u,
  p.extinction_g,
  p.extinction_r,
  p.extinction_i,
  p.extinction_z,
  p.petroR50_u,
  p.petroR50_g,
  p.petroR50_r,
  p.petroR50_i,
  p.petroR50_z,
  f.w1_nanomaggies,
  f.w1_nanomaggies_ivar,
  f.w2_nanomaggies,
  f.w2_nanomaggies_ivar
INTO mydb.redmapper_sdssWISEphot from mydb.redmapper_newobjid as m
  JOIN dr14.PhotoObj AS p ON m.redmapper_objid=p.objid
  JOIN dr14.wiseForcedTarget AS f ON m.redmapper_objid=f.objid
ORDER BY m.redmapper_id
```

In [19]:
finalcasfile = os.path.join( os.getenv('REDMAPPER_DIR'), rmversion, 'casjobs', 
                       'redmapper_newobjid.txt'.format(rmversion) )
if os.path.isfile(finalcasfile):
    print('CasJobs file {} exists.'.format(finalcasfile))
else:
    print('Writing {}'.format(finalcasfile))
    np.savetxt( finalcasfile, np.vstack( [ [rm['MEM_MATCH_ID']], [newobjid] ] ).T, 
               header='redmapper_id redmapper_objid', fmt=('%i', '%i') )

CasJobs file /Users/ioannis/research/data/redmapper/v6.3.1/casjobs/redmapper_newobjid.txt exists.


### Construct the final output table.

Finally, we put all the pieces together.  We read the output FITS table (*redmapper_sdssWISEphot.fits*) from CasJobs, collect the updated coordinates and ObjIDs, convert the various SDSS magnitudes and unWISE fluxes into *maggies*, and write out the final photometric table.

In [20]:
inphotfile = os.path.join( os.getenv('REDMAPPER_DIR'), rmversion, 'casjobs',
                          'redmapper_sdssWISEphot.fits' )
inphot = Table(fitsio.read(inphotfile, ext=1, upper=True))
inphot

REDMAPPER_ID,REDMAPPER_OBJID,OBJID,RA,DEC,RUN,RERUN,CAMCOL,FIELD,TYPE,PETROFLUX_U,PETROFLUX_G,PETROFLUX_R,PETROFLUX_I,PETROFLUX_Z,PETROFLUXIVAR_U,PETROFLUXIVAR_G,PETROFLUXIVAR_R,PETROFLUXIVAR_I,PETROFLUXIVAR_Z,MODELFLUX_U,MODELFLUX_G,MODELFLUX_R,MODELFLUX_I,MODELFLUX_Z,MODELFLUXIVAR_U,MODELFLUXIVAR_G,MODELFLUXIVAR_R,MODELFLUXIVAR_I,MODELFLUXIVAR_Z,CMODELFLUX_U,CMODELFLUX_G,CMODELFLUX_R,CMODELFLUX_I,CMODELFLUX_Z,CMODELFLUXIVAR_U,CMODELFLUXIVAR_G,CMODELFLUXIVAR_R,CMODELFLUXIVAR_I,CMODELFLUXIVAR_Z,EXTINCTION_U,EXTINCTION_G,EXTINCTION_R,EXTINCTION_I,EXTINCTION_Z,PETROR50_U,PETROR50_G,PETROR50_R,PETROR50_I,PETROR50_Z,W1_NANOMAGGIES,W1_NANOMAGGIES_IVAR,W2_NANOMAGGIES,W2_NANOMAGGIES_IVAR
int32,int64,int64,float64,float64,int16,int16,uint8,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
1,1237662340012638224,1237662340012638224,239.583335437,27.2334095464,3927,301,2,85,3,95.8095,548.691,1765.14,2428.82,4284.27,0.000654984,1.53766e-05,4.45877e-06,1.67189e-06,1.62822e-06,62.9277,585.716,1550.64,2355.87,3331.97,0.121141,0.239921,0.066257,0.0291726,0.00400068,29.5241,562.623,1550.71,2354.2,3215.09,0.532083,0.260197,0.0662513,0.0292167,0.00430256,0.187233,0.145891,0.100927,0.0749992,0.0557857,101.315,14.1118,19.4193,16.3806,25.6271,39647.6,0.000414198,42470.3,4.17227e-05
2,1237651715872325879,1237651715872325879,250.082552977,46.7114748935,1453,301,5,57,3,19.338,36.6594,162.105,247.893,200.168,0.0739838,0.151757,0.02356,0.00682604,0.000865781,10.7362,59.429,227.749,364.422,460.244,0.486459,1.63991,0.470184,0.166813,0.0149121,31.4261,44.5324,187.134,291.81,274.856,0.074432,2.72901,0.650965,0.243251,0.0401029,0.102034,0.0795039,0.0550005,0.0408712,0.0304007,10.4942,3.68733,4.49805,4.2099,2.44081,7234.43,0.00300798,8665.86,0.000288756
3,1237655500272500810,1237655500272500810,197.872956459,-1.3411176982,2334,301,6,13,3,-19.8656,115.426,378.889,580.535,108.208,0.00854354,0.000192759,3.57353e-05,2.77351e-05,4.0306e-05,5.86714,157.761,493.495,743.643,920.437,0.139,0.461119,0.159731,0.0754257,0.00699053,1.85612,139.596,457.745,655.826,464.897,1.89186,0.552198,0.178729,0.0908235,0.026475,0.114984,0.0895949,0.0619813,0.0460588,0.0342593,-9999.0,6.99923,7.68177,7.68803,1.21615,12584.4,0.000359241,14754.4,3.38049e-05
4,1237664852576633380,1237664852576633380,230.600086411,27.7143712172,4512,301,2,209,3,53.7736,403.387,973.158,1473.43,1718.61,0.0553758,0.0757284,0.0124448,0.00442942,0.00150141,65.9349,430.382,1042.85,1577.12,2106.42,0.436373,0.616532,0.167542,0.0743046,0.0138082,57.1911,424.72,1042.89,1546.52,1808.63,0.413412,0.633072,0.167529,0.0772798,0.0188815,0.168863,0.131577,0.0910242,0.0676408,0.0503123,4.16933,5.22759,5.23404,5.24658,4.07635,24571.6,0.00175937,25315.6,0.000159679
5,1237671260126576920,1237671260126576920,137.300746077,10.9735952074,6004,301,1,142,3,-0.00106838,41.8442,133.936,319.034,329.451,0.0257284,0.0192987,0.00147911,0.000736845,0.000321718,-0.529659,47.2544,120.496,224.724,287.634,0.341791,0.984224,0.43414,0.202047,0.0175966,-0.0337152,48.1884,151.46,373.459,369.811,11.2687,0.888147,0.242618,0.068772,0.0097743,0.181033,0.14106,0.0975847,0.0725159,0.0539385,-9999.0,6.88205,8.34846,11.3614,9.01258,3657.63,0.000374248,3202.04,4.0285e-05
6,1237662306722447498,1237662306722447498,203.833732496,41.0011448391,3919,301,4,21,3,24.0227,56.5474,264.676,400.319,415.4,0.0485586,0.397669,0.0294581,0.0130675,0.00244697,9.68235,66.951,276.381,447.888,615.01,1.07751,3.1378,0.761756,0.313797,0.0245817,41.4308,61.1556,276.384,418.759,486.09,0.0785992,3.77051,0.761738,0.359596,0.0399041,0.0382186,0.0297797,0.0206014,0.0153091,0.0113871,11.639,3.59104,4.29219,3.8044,2.58524,10416.5,0.00180828,13191.3,0.000162028
7,12376786023823447

In [21]:
match = np.in1d(rm['MEM_MATCH_ID'], inphot['REDMAPPER_ID'])
matchindx = np.where(match)[0]
missindx = np.where(~match)[0]

#### Initialize the output table.

Note that we have to convert ObjID to run, rerun, camcol, and field ourselves because *inphot* is missing 4 objects.

Also note that we use the (updated) RA,Dec coordinates from our CasJobs query, as a small fraction of objects (of order ~50) changed coordinates by more than 1 arcsec (and some by as much as 10 arcsec!).

In [22]:
newrun, newrerun, newcamcol, newfield, newid = unpack_objid(newobjid)
assert( np.sum(newrun[matchindx] - inphot['RUN']) == 0)
assert( np.sum(newrerun[matchindx] - inphot['RERUN']) == 0)
assert( np.sum(newcamcol[matchindx] - inphot['CAMCOL']) == 0)
assert( np.sum(newfield[matchindx] - inphot['FIELD']) == 0)

In [23]:
outtable = Table()
outtable.add_column(Column(data=rm['MEM_MATCH_ID']))
outtable.add_column(Column(data=newobjid, name='OBJID'))
outtable.add_column(Column(data=newrun, name='RUN'))
outtable.add_column(Column(data=newrerun, name='RERUN'))
outtable.add_column(Column(data=newcamcol, name='CAMCOL'))
outtable.add_column(Column(data=newfield, name='FIELD'))
outtable.add_column(Column(name='RA', dtype='f8', length=nobj, unit='deg'))
outtable.add_column(Column(name='DEC', dtype='f8', length=nobj, unit='deg'))
# outtable.add_column(Column(data=rm['RA'], name='REDMAPPER_RA', unit='deg'))
# outtable.add_column(Column(data=rm['DEC'], name='REDMAPPER_DEC', unit='deg'))
# outtable.add_column(Column(data=objid, name='REDMAPPER_OBJID'))
for col in ('PETROMAGGIES', 'MODELMAGGIES', 'CMODELMAGGIES'):
    outtable.add_column(Column(name=col, dtype='f4', shape=(5,), length=nobj))
    outtable.add_column(Column(name=col+'_IVAR', dtype='f4', shape=(5,), length=nobj))
outtable.add_column(Column(name='WISE_NANOMAGGIES', dtype='f4', shape=(2,), length=nobj))
outtable.add_column(Column(name='WISE_NANOMAGGIES_IVAR', dtype='f4', shape=(2,), length=nobj))
for col in ('EXTINCTION', 'PETROR50'):
    outtable.add_column(Column(name=col, dtype='f4', shape=(5,), length=nobj))

In [24]:
for col in ('RA', 'DEC'):
    outtable[col][matchindx] = inphot[col]
    outtable[col][missindx] = rm[col][missindx]
for col in ('PETROFLUX', 'MODELFLUX', 'CMODELFLUX', 'EXTINCTION', 'PETROR50'):
    for ib, band in enumerate( ['U', 'G', 'R', 'I', 'Z'] ):
        outtable[col.replace('FLUX', 'MAGGIES')][matchindx, ib] = inphot[col+'_'+band]
        if 'FLUX' in col:
            outtable[col.replace('FLUX', 'MAGGIES_IVAR')][matchindx, ib] = inphot[col+'IVAR_'+band]
outtable['WISE_NANOMAGGIES'][matchindx, :] = np.vstack( (inphot['W1_NANOMAGGIES'], 
                                                         inphot['W2_NANOMAGGIES']) ).T
outtable['WISE_NANOMAGGIES_IVAR'][matchindx, :] = np.vstack( (inphot['W1_NANOMAGGIES_IVAR'], 
                                                              inphot['W2_NANOMAGGIES_IVAR']) ).T

In [25]:
outtable

MEM_MATCH_ID,OBJID,RUN,RERUN,CAMCOL,FIELD,RA,DEC,PETROMAGGIES [5],PETROMAGGIES_IVAR [5],MODELMAGGIES [5],MODELMAGGIES_IVAR [5],CMODELMAGGIES [5],CMODELMAGGIES_IVAR [5],WISE_NANOMAGGIES [2],WISE_NANOMAGGIES_IVAR [2],EXTINCTION [5],PETROR50 [5]
,,,,,,deg,deg,,,,,,,,,,
int32,int64,int64,int64,int64,int64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
1,1237662340012638224,3927,301,2,85,239.583335437,27.2334095464,95.8095 .. 4284.27,0.000654984 .. 1.62822e-06,62.9277 .. 3331.97,0.121141 .. 0.00400068,29.5241 .. 3215.09,0.532083 .. 0.00430256,39647.6 .. 42470.3,0.000414198 .. 4.17227e-05,0.187233 .. 0.0557857,101.315 .. 25.6271
2,1237651715872325879,1453,301,5,57,250.082552977,46.7114748935,19.338 .. 200.168,0.0739838 .. 0.000865781,10.7362 .. 460.244,0.486459 .. 0.0149121,31.4261 .. 274.856,0.074432 .. 0.0401029,7234.43 .. 8665.86,0.00300798 .. 0.000288756,0.102034 .. 0.0304007,10.4942 .. 2.44081
3,1237655500272500810,2334,301,6,13,197.872956459,-1.3411176982,-19.8656 .. 108.208,0.00854354 .. 4.0306e-05,5.86714 .. 920.437,0.139 .. 0.00699053,1.85612 .. 464.897,1.89186 .. 0.026475,12584.4 .. 14754.4,0.000359241 .. 3.38049e-05,0.114984 .. 0.0342593,-9999.0 .. 1.21615
4,1237664852576633380,4512,301,2,209,230.600086411,27.7143712172,53.7736 .. 1718.61,0.0553758 .. 0.00150141,65.9349 .. 2106.42,0.436373 .. 0.0138082,57.1911 .. 1808.63,0.413412 .. 0.0188815,24571.6 .. 25315.6,0.00175937 .. 0.000159679,0.168863 .. 0.0503123,4.16933 .. 4.07635
5,1237671260126576920,6004,301,1,142,137.300746077,10.9735952074,-0.00106838 .. 329.451,0.0257284 .. 0.000321718,-0.529659 .. 287.634,0.341791 .. 0.0175966,-0.0337152 .. 369.811,11.2687 .. 0.0097743,3657.63 .. 3202.04,0.000374248 .. 4.0285e-05,0.181033 .. 0.0539385,-9999.0 .. 9.01258
6,1237662306722447498,3919,301,4,21,203.833732496,41.0011448391,24.0227 .. 415.4,0.0485586 .. 0.00244697,9.68235 .. 615.01,1.07751 .. 0.0245817,41.4308 .. 486.09,0.0785992 .. 0.0399041,10416.5 .. 13191.3,0.00180828 .. 0.000162028,0.0382186 .. 0.0113871,11.639 .. 2.58524
7,1237678602382344780,7713,301,5,282,8.03426642874,18.1070248394,8.04202 .. 48.6239,0.196601 .. 0.0139645,1.35466 .. 84.9651,2.34027 .. 0.106668,28.9637 .. 64.3609,0.0334751 .. 0.186488,2258.34 .. 2754.64,0.00228424 .. 0.000181519,0.53113 .. 0.158249,6.93591 .. 1.18464
8,1237652599036837935,1659,301,2,238,340.832481336,-9.5919218039,12.0261 .. 45.0043,0.178021 .. 0.0112217,-0.0417006 .. 30.5261,2.21311 .. 0.178466,2.26608 .. 52.2221,0.0365428 .. 0.0556722,1232.33 .. 1192.63,0.00189179 .. 0.000157633,0.200557 .. 0.0597554,8.17259 .. 2.79073


In [26]:
outphotfile = os.path.join( os.getenv('REDMAPPER_DIR'), rmversion, 
                           'redmapper-{}-lgt5-sdssWISEphot.fits'.format(rmversion) )
if os.path.isfile(outphotfile):
    print('Output photometry file {} exists.'.format(outphotfile))
else:
    print('Writing {}'.format(outphotfile))
    outtable.write(outphotfile)

Writing /Users/ioannis/research/data/redmapper/v6.3.1/redmapper-v6.3.1-sdssWISEphot-dr14.fits
